### Bayesian Network for Mutation Impact Prediction

**Course**: CSE 150A/250A \
**Team Name:** SeqPredict \
**Team Members:** Qiwen Xu, Michael Kroyan, Janice Rincon, Vibusha Vadivel, Jiya Makhija

In [ ]:
pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 23.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re

In [ ]:

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/cse150a_finalproject/variant_summary.txt.gz', sep='\t', compression='gzip')

print(f"Total variants: {len(df)}")

HTTPError: HTTP Error 401: Unauthorized

In [ ]:
df.head()

NameError: name 'df' is not defined

### Feature Engineering

In [ ]:
df['ClinSigSimple'].unique()

In [ ]:
"""
ClinSigSimple          integer, 0 = no current value of Likely pathogenic; Pathogenic; Likely pathogenic, low penetrance;
                                    Pathogenic, low penetrance; Likely risk allele; or Risk allele
                                1 = at least one current record submitted with an interpretation of Likely pathogenic; Pathogenic;
                                    Likely pathogenic, low penetrance; Pathogenic, low penetrance; Likely risk allele;
                                    or Risk allele (independent of whether that record includes assertion criteria and evidence).
                               -1 = no values for clinical significance at all for this variant or set of variants; used for
                                    the "included" variants that are only in ClinVar because they are included in a
                                    haplotype or genotype with an interpretation


Removing -1 as a value since ClinVar documentation says it's for variants
that don't have pathogenicity classifications and have no clinical signficance
"""

df = df[df['ClinSigSimple'].isin([0, 1])].copy()



In [ ]:
# remove missing data
df = df.dropna(subset=['Type', 'Chromosome', 'Start', 'Stop', 'GeneSymbol'])

In [ ]:
df['Type'].unique()

In [ ]:
# per our project plan
features = pd.DataFrame()


# feature 1 - VariantType
variant_mapping = {
    'single nucleotide variant': 'SNV',
    'Deletion': 'Deletion',
    'Insertion': 'Indel',
    'Indel': 'Indel',
    'Duplication': 'Indel',
    'copy number gain': 'CNV',
    'copy number loss': 'CNV',
    'Microsatellite': 'Other',
    'Variation': 'Other',
    'Complex': 'Other',
    'Translocation': 'Other',
    'Inversion': 'Other',
    'fusion': 'Other',
    'protein only': 'Other',
    'Tandem duplication': 'Other'
}

features['VariantType'] = df['Type'].map(variant_mapping)


In [ ]:
# feature 2 - LengthChange_bin
"""Indels = Insertions + Deletions
Insertions: Extra DNA gets added
Deletions: DNA gets removed

https://pmc.ncbi.nlm.nih.gov/articles/PMC5570956/#Sec1 :
"A rate of 2.94 indels (1-20 bp) and
0.16 structural variants (>20 bp) per generation was estimated based on whole genome sequencing of 250 families"

so single nucleotide variant is 1 small indels (2-20) and large >20 - so large indels are more likely to cause disease because rare?
"""

variant_length = df['Stop'] - df['Start'] + 1

features['LengthChange_bin'] = pd.cut(variant_length, bins=[-1, 1, 20, float('inf')], labels=['SNV', 'small_indel', 'large_indel'])

In [ ]:
# feature 3 - Frameshift
"""removing indels not divisible by 3 - because they cause frameshifts

NOTE: this is an approximation because ClinVar provides genomic alleles, not transcript-specific coding alleles

"""
ref_len = df["ReferenceAllele"].astype(str).str.len()
alt_len = df["AlternateAllele"].astype(str).str.len()

length_change = alt_len - ref_len   # insertion = positive, deletion = negative

frameshift = (length_change % 3 != 0) & df["Type"].isin(["Deletion", "Insertion", "Indel"])
features['Frameshift'] = frameshift.astype(int)

In [ ]:
# feature 4 - StopGain
"""StopGain is caused by a single DNA letter change that accidentally creates a STOP signal
NOTE: this is an approximation because True stop-gain classification depends on transcript-specific coding
context, which is not available in our dataset.

Approximation stop-gain mutations by parsing the HGVS protein notation (like '(p.Gln232Ter)', '(p.Arg33*)')
or text descriptions such as 'nonsense mutation' or 'stop-gain'"""

stop_pattern = re.compile(
    r"\(p\.[A-Za-z]{3}\d+(ter|\*)\)", re.IGNORECASE
)

def is_stopgain(name):
    if pd.isna(name):
        return False
    s = str(name)

    # protein-based stop-gain
    if stop_pattern.search(s):
        return True

    # text-based descriptions
    s_low = s.lower()
    text_stop_indicators = ['nonsense mutation', 'stop-gain', 'stop_gain','stopgained']
    # 1 if variant creates a premature STOP codon, 0 if not
    return any(indicator in s_low for indicator in text_stop_indicators)

features['StopGain'] = df['Name'].apply(is_stopgain).astype(int)

In [ ]:
df['GeneSymbol'].value_counts()

In [ ]:
# feature 5 - GeneGroup
gene_mapping = {
    'TTN': 'structural',           # Titin - structural muscle protein
    'BRCA2': 'tumor_suppressor',
    'ATM': 'tumor_suppressor',
    'APC': 'tumor_suppressor',
    'NF1': 'tumor_suppressor',    # Neurofibromatosis

    # Other common tumor suppressors
    'BRCA1': 'tumor_suppressor', 'TP53': 'tumor_suppressor',
    'PTEN': 'tumor_suppressor', 'VHL': 'tumor_suppressor',
    'RB1': 'tumor_suppressor', 'MLH1': 'tumor_suppressor',
    'MSH2': 'tumor_suppressor', 'MSH6': 'tumor_suppressor',
    'PMS2': 'tumor_suppressor', 'PALB2': 'tumor_suppressor',

    # Metabolism
    'APOE': 'metabolism', 'LDLR': 'metabolism', 'GBA': 'metabolism',
    'PAH': 'metabolism', 'HFE': 'metabolism',

    # Ion channels
    'CFTR': 'channel_protein', 'SCN5A': 'channel_protein',
    'SCN1A': 'channel_protein', 'KCNQ1': 'channel_protein',

    # Structural
    'FBN1': 'structural', 'COL1A1': 'structural', 'COL1A2': 'structural',
    'MYH7': 'structural', 'MYBPC3': 'structural',

    # DNA repair
    'RNASEL': 'dna_repair', 'MUTYH': 'dna_repair'
}

features['GeneGroup'] = df['GeneSymbol'].map(gene_mapping).fillna('other')

In [ ]:
df['Chromosome'].value_counts()

In [ ]:
# feature 6 - Chromosome_clean
def clean_chromosome(chrom_value):
    chrom_str = str(chrom_value).strip()

    if chrom_str in ['X', 'Y']:
        return 'sex_chr'
    elif chrom_str == 'MT':
        return 'mitochondrial'
    # missing/unknown
    elif chrom_str in ['na', 'Un', 'nan']:
        return 'unknown'
    # everything else is autosome from 1-22 (autosomes are regular chromosomes)
    else:
        return 'autosome'

features['Chromosome_clean'] = df['Chromosome'].apply(clean_chromosome)


In [ ]:
print(features['Chromosome_clean'].value_counts())

In [ ]:
# feature 7 - PositionBin
"""some regions are gene-rich vs gene-poor - may influence the pathogenicity"""
features['PositionBin'] = pd.qcut(df['Start'], q=3,
                                 labels=['low', 'mid', 'high'],
                                 duplicates='drop')


In [ ]:
# feature 8 - PhenotypeCount_bin
"""variants linked to many diseases are more likely to be genuinely pathogenic

none: about 3000 variants (0 diseases)
few: about 7.7 million variants (1-2 diseases)
moderate: about 134,000 variants (3-5 diseases)
many: about 3,000 variants (6+ diseases)

"""

def count_diseases(pheno_list):
    if pd.isna(pheno_list) or pheno_list == '-':
        return 0
    return len(str(pheno_list).split(';'))

disease_counts = df['PhenotypeList'].apply(count_diseases)
features['PhenotypeCount_bin'] = pd.cut(disease_counts,
                                        bins=[-1, 0, 2, 5, float('inf')],
                                        labels=['none', 'few', 'moderate', 'many'])


In [ ]:
print(disease_counts.value_counts().sort_index())


In [ ]:
# feature 9 - ClinSigSimple_num (target variable)
"""What our BN will predict - Binary classification target: 0=benign, 1=pathogenic"""
features['ClinSigSimple_num'] = df['ClinSigSimple']


In [ ]:
target_counts = features['ClinSigSimple_num'].value_counts()
target_counts

In [ ]:
print(features['ClinSigSimple_num'].value_counts(normalize=True) * 100)

In [ ]:
features.to_csv('features_clean.csv', index=False)

### Model creation

In [ ]:
# load cleaned/processed features
data = pd.read_csv('/content/drive/MyDrive/cse150a_finalproject/features_clean.csv')

len(data)

7867151

In [ ]:
data.head()

,VariantType,LengthChange_bin,Frameshift,GeneGroup,Chromosome_clean,PositionBin,PhenotypeCount_bin,ClinSigSimple_num,StopGain
0,Indel,small_indel,0,other,autosome,low,few,1,0
1,Indel,small_indel,0,other,autosome,low,few,1,0
2,Deletion,small_indel,0,other,autosome,low,few,1,0
3,Deletion,small_indel,0,other,autosome,low,few,1,0
4,SNV,SNV,0,other,autosome,mid,few,0,0


In [ ]:
data_clean = data.drop(columns=["PhenotypeCount_bin","StopGain", "Frameshift", "LengthChange_bin", "PositionBin"])

In [ ]:
data_clean.head()

,VariantType,GeneGroup,Chromosome_clean,ClinSigSimple_num
0,Indel,other,autosome,1
1,Indel,other,autosome,1
2,Deletion,other,autosome,1
3,Deletion,other,autosome,1
4,SNV,other,autosome,0


In [ ]:
data_clean["ClinSigSimple_num"].value_counts(normalize=True)

,proportion
ClinSigSimple_num,
0,0.905749
1,0.094251


In [ ]:
from sklearn.model_selection import train_test_split

# split data into train-test
train_df, test_df = train_test_split(data_clean, test_size=0.2, random_state=42)

In [ ]:
# learn the DAG from train with HillClimbSearch
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.estimators import HillClimbSearch, BIC, ExpertKnowledge
from pgmpy.inference import VariableElimination
from pgmpy.estimators import MaximumLikelihoodEstimator
from sklearn.metrics import accuracy_score
import numpy as np

TARGET = "ClinSigSimple_num"
feature_cols = [c for c in data_clean.columns if c != TARGET]

# Forbid edges going *out* of the label
forbidden = [(TARGET, f) for f in feature_cols]

expert_knowledge = ExpertKnowledge(
    forbidden_edges=forbidden
)

hc = HillClimbSearch(train_df)

dag = hc.estimate(
   scoring_method=BIC(train_df),
   max_indegree=4,          # limit parents per node
   expert_knowledge=expert_knowledge,
)

print("Learned edges:")
print(list(dag.edges()))

  0%|          | 0/1000000 [00:00<?, ?it/s]

Learned edges:
[('VariantType', 'ClinSigSimple_num'), ('VariantType', 'GeneGroup'), ('VariantType', 'Chromosome_clean'), ('GeneGroup', 'ClinSigSimple_num'), ('Chromosome_clean', 'GeneGroup'), ('Chromosome_clean', 'ClinSigSimple_num')]


In [ ]:
# build a BayesianNetwork and learn the Conditional Probability Distribitions (CPDs) -> MLE

# build the BN using the learned DAG edges
model = DiscreteBayesianNetwork(dag.edges())

# fit CPDs using MLE
model.fit(train_df, estimator=MaximumLikelihoodEstimator)

# inspect learnt CPDs
for cpd in model.get_cpds():
  print(cpd)

+-----------------------+------------+
| VariantType(CNV)      | 0.00779158 |
+-----------------------+------------+
| VariantType(Deletion) | 0.039634   |
+-----------------------+------------+
| VariantType(Indel)    | 0.0258756  |
+-----------------------+------------+
| VariantType(Other)    | 0.00996708 |
+-----------------------+------------+
| VariantType(SNV)      | 0.916732   |
+-----------------------+------------+
+----------------------+-----+-----------------------------+
| Chromosome_clean     | ... | Chromosome_clean(unknown)   |
+----------------------+-----+-----------------------------+
| GeneGroup            | ... | GeneGroup(tumor_suppressor) |
+----------------------+-----+-----------------------------+
| VariantType          | ... | VariantType(SNV)            |
+----------------------+-----+-----------------------------+
| ClinSigSimple_num(0) | ... | 0.7272727272727273          |
+----------------------+-----+-----------------------------+
| ClinSigSimple_num(1)

In [ ]:
# Use inference for prediction on test set
from pgmpy.inference import VariableElimination
import numpy as np

infer = VariableElimination(model)

TARGET = "ClinSigSimple_num"
feature_cols = [c for c in data_clean.columns if c != TARGET]

y_true = []
y_pred = []
y_proba = []

for _, row in test_df.iterrows():
    # evidence excludes the target
    evidence = {col: row[col] for col in feature_cols}

    # BN inference
    q = infer.query(variables=[TARGET], evidence=evidence, show_progress=False)

    # Probability of class 1 (pathogenic)
    prob_1 = q.values[1]     # q.values is array: [P(0), P(1)]
    y_proba.append(prob_1)

    # Argmax predicted class
    idx = q.values.argmax()  # returns index 0 or 1
    pred_state = q.state_names[TARGET][idx]
    y_pred.append(pred_state)

    # True label
    y_true.append(row[TARGET])

# Convert to arrays for sklearn
y_true = np.array(y_true, dtype=int)
y_pred = np.array(y_pred, dtype=int)
y_proba = np.array(y_proba, dtype=float)


In [ ]:
# TESTING METRICS
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve, average_precision_score

# Compute accuracy
accuracy = np.mean(np.array(y_true) == np.array(y_pred))
print("Test accuracy:", accuracy)

# Classification report (precision/recall/F1)
print("Classification: ")
print(classification_report(y_true, y_pred, digits=4))

# ROC AUC
roc_auc = roc_auc_score(y_true, y_proba)
print(f"ROC AUC: {roc_auc:.4f}")

# Precision–Recall curve
prec, rec, thresholds = precision_recall_curve(y_true, y_proba)

# Average precision (PR-AUC)
ap = average_precision_score(y_true, y_proba)
print(f"Average Precision (PR AUC): {ap:.4f}")


Test accuracy: 0.9169807891162688
Classification: 
              precision    recall  f1-score   support

           0     0.9310    0.9810    0.9554   1424946
           1     0.6240    0.3027    0.4076    148485

    accuracy                         0.9170   1573431
   macro avg     0.7775    0.6418    0.6815   1573431
weighted avg     0.9021    0.9170    0.9037   1573431

ROC AUC: 0.7372
Average Precision (PR AUC): 0.3765
